In [ ]:
!pip install scikit-learn==0.23.2
!pip install sklearn_crfsuite
!pip install pythainlp


     |████████████████████████████████| 6.8 MB 4.3 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [ ]:
import sklearn_crfsuite
import pandas as pd
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from pythainlp.tag import pos_tag_sents

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/superai/train_val_lst20_rule_based.csv")
test = pd.read_csv("/content/drive/MyDrive/superai/test_lst20_rule_based.csv")

In [ ]:
train = train.groupby('sentence_id').agg(lambda x : list(x))
test = test.groupby('sentence_id').agg(lambda x : list(x))


In [ ]:
train["POS"] = pos_tag_sents(list(train["words"]))
test["POS"] = pos_tag_sents(list(test["words"]))


In [ ]:
train["newPOS"] = train["POS"].apply(lambda x : [i[1] for i in x])
test["newPOS"] = test["POS"].apply(lambda x : [i[1] for i in x])


In [ ]:
_NER_TAGS = [
        "O",
        "B_BRN",
        "B_DES",
        "B_DTM",
        "B_LOC",
        "B_MEA",
        "B_NUM",
        "B_ORG",
        "B_PER",
        "B_TRM",
        "B_TTL",
        "I_BRN",
        "I_DES",
        "I_DTM",
        "I_LOC",
        "I_MEA",
        "I_NUM",
        "I_ORG",
        "I_PER",
        "I_TRM",
        "I_TTL",
        "E_BRN",
        "E_DES",
        "E_DTM",
        "E_LOC",
        "E_MEA",
        "E_NUM",
        "E_ORG",
        "E_PER",
        "E_TRM",
        "E_TTL",
    ]

In [ ]:
train_list = [ [ [train["words"][i][j] , train["newPOS"][i][j] , train["labels"][i][j] ] for j in range(len(train["words"][i]))]   for i in range(len(train))]
test_list = [ [ [test["words"][i][j] , test["newPOS"][i][j] , test["labels"][i][j]] for j in range(len(test["words"][i]))]   for i in range(len(test))]


In [ ]:
def word2features(sent, i):
    
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word': word,
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:postag': postag1,
            '-1:word': word1,
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag1,
            '+1:word': word1,
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
train.to_csv("train.csv",index=False)
test.to_csv("test.csv",index=False)

In [ ]:
X_train = [sent2features(s) for s in train_list]
y_train = [sent2labels(s) for s in train_list]

X_test = [sent2features(s) for s in test_list]
y_test = [sent2labels(s) for s in test_list]

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=_NER_TAGS)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.944462796122691

In [ ]:
import pickle

# save
with open('crfmodel.pkl','wb') as f:
    pickle.dump(crf,f)

In [ ]:
f=open('/content/ne_test.txt','r')
text = f.readlines()
input_text = []

for i in range(len(text)):
  if text[i] != "\n":
    input_text.append(text[i].replace(" ","_").replace(u"\xa0","_").replace("\n","")) #\xa0 คือ space ใน utf8 
  else:
    input_text.append("_")

input_text = input_text[:len(input_text)-1] # ตัด \n ท้ายไฟล์ทิ้ง

# Predict

In [ ]:
from pythainlp.tokenize import clause_tokenize

kaggle_clause = clause_tokenize(input_text)


Corpus: lst20-cls
- Downloading: lst20-cls 0.2


100%|██████████| 3738912/3738912 [00:00<00:00, 45912186.03it/s]


In [ ]:
kaggle_pos = [  [ tag[1] for tag in clause] for clause in pos_tag_sents(kaggle_clause)]
kaggle_list = [ [ [kaggle_clause[i][j] , kaggle_pos[i][j]  ] for j in range(len(kaggle_clause[i]))]   for i in range(len(kaggle_clause))]


In [ ]:
kaggle_input = [sent2features(s) for s in kaggle_list]



In [ ]:
kaggle_pred = crf.predict(kaggle_input)
kaggle_pred

In [ ]:
len(kaggle_pred)

3648

In [ ]:
prediction=[]
for clause_pred in kaggle_pred:
  prediction+=clause_pred
len(prediction)

69561

In [ ]:
kaggle_input

In [ ]:
id = [i+1 for i in range(len(prediction))]
submission = pd.DataFrame({"Id":id,"Predicted":prediction})

In [ ]:
submission

In [ ]:
submission.to_csv("submission_crf_post.csv",index=False)

In [ ]:
pos_tag = []
for pos in kaggle_pos:
  pos_tag+=pos
len(pos_tag)

In [ ]:
# for i in range(len(prediction)):
#   if pos_tag[i] == "NTTL" and input_text[i]!="_":
#     prediction[i]="B_TTL"

In [ ]:
# for i in range(1,len(prediction)):

#     if prediction[i-1][0] == "I" and prediction[i][0]=="O":
#       if input_text[i-1] == "_":

#         if (i+2<len(prediction) and prediction[i+2][0] in ["E","I"]) or input_text[i]=="_":
#           prediction[i]="I"+prediction[i-1][1:]
#         else :
#           prediction[i]="E"+prediction[i-1][1:] #can be B

#       else :

#         if i-2>=0 and prediction[i-2][0] == "I":
#           prediction[i-1]="E"+prediction[i-1][1:]
#         else :
#           prediction[i-1]="B"+prediction[i-1][1:]

#     elif prediction[i-1][0] == "O" and prediction[i][0]=="I":
#       if input_text[i] == "_":
#         if (i-2>=0 and prediction[i-2][0] == "I") or input_text[i-1]=="_":
#           prediction[i-1]="I"+prediction[i][1:]
#         else :
#           prediction[i-1]="B"+prediction[i][1:]
#       else:
#         prediction[i]="B"+prediction[i][1:]

#     elif prediction[i-1][0] == "E" and prediction[i][0]=="I":
#       if input_text[i] == "_":
#         if (i-2>=0 and prediction[i-2][0] in ["I","B"]) or input_text[i-1]=="_":
#           prediction[i-1]="I"+prediction[i][1:]
#         else :
#           prediction[i-1]="B"+prediction[i][1:]
#       else:
#         if i-2>=0 and prediction[i-2][0] == "I":
#           prediction[i-1]="I"+prediction[i-1][1:]
#         else :
#           prediction[i-1]="B"+prediction[i-1][1:]
    
#     elif prediction[i-1][0] == "O" and prediction[i][0]=="E":
#       prediction[i]="B"+prediction[i][1:]

#     elif prediction[i-1][0] == "E" and prediction[i][0]=="E":
#       if i-2>=0 and prediction[i-2][0] == "I":
#         prediction[i-1]="I"+prediction[i-1][1:]
#       else :
#         prediction[i]="B"+prediction[i][1:]
    
#     elif prediction[i-1][0]=="I" and prediction[i][0]=="B":
#       if i+1<len(prediction) and prediction[i+1][0]=="E":
#         prediction[i]="I"+prediction[i-1][1:]
#       else:
#         prediction[i]="E"+prediction[i][1:]


In [ ]:
pred = pd.read_csv("/content/submission_pure_crf.csv")["Predicted"]
post_pred = pd.read_csv("/content/submission_crf_post.csv")["Predicted"]

In [ ]:
y_true = []
y_model = []

for i in range(len(y_test)):
  y_true+=y_test[i]
  y_model+=y_pred[i]

In [ ]:
y_true

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_model))